## In this Notebook, we will create a basic Q&A chain and Agent over a SQL database

### Architecture

At a high-level, the steps of any SQL chain and agent are:

- `Convert question to SQL query using the LLM Model.`
- `Execute the SQL query.`
- `getting the Model to respond to user input using the query results.`

In [4]:
%pip install langchain_experimental
%pip install psycopg2
%pip install langchain_openai


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 11.1 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
import psycopg2
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [9]:
llm = ChatOpenAI(temperature=0)
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
#chain = create_sql_query_chain(llm, db)



prompt_template = PromptTemplate.from_template(
    """You Answer the question based on the context below.

Context:
    In the database, there are entities,gdapi_entities, datasets,dataset_tables,entity_data tables. Use Below relationships. 
    entities table Stores name of customer.
    gdapi_entities is a link between entities table and datasets. gdapi_entities id is used as a foreign key by datasets table in entity_id column.
    dataset_tables has a column dataset_id this is a foreign key on datasets table on id column. dataset_tables has a column with name as table_name .
    entity_data table has the size of data stored for each entity . this table also has the table_name column.
    Use the table_name column in dataset_tables and entity_data for the join between two tables.
    
Question: """
)
prompt = prompt_template.format()

# llm.invoke(prompt)


In [12]:
from html import entities


host = 'localhost'
port = '5432'
username = 'postgres'
password = 'password'
database_schema = 'delphis'
#mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"
postgres_uri = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database_schema}"
# postgres_uri = f"postgresql+psycopg2://{username}@{host}:{port}/{database_schema}"

# Create a temporary SQLDatabase object to check available tables
temp_db = SQLDatabase.from_uri(postgres_uri)
available_tables = temp_db.get_usable_table_names()
print("Available tables:", available_tables)

# Check if the required tables exist in the database
required_tables = ["entities", "gdapi_entities", "datasets", "dataset_tables", "entity_data"]


# Create the SQLDatabase object with the required tables
db = SQLDatabase.from_uri(postgres_uri, include_tables=required_tables, sample_rows_in_table_info=2)

chain = create_sql_query_chain(llm, db)
context = """
In the database, there are entities, gdapi_entities, datasets, dataset_tables, entity_data tables. Use the relationships below:
- entities table stores the name of the customer. 
- gdapi_entities is a link between entities table and datasets.
- gdapi_entities has a column named as delphius_id, use delphius_id column to join with id column of entity table.
- gdapi_entities id is used as a foreign key by datasets table in entity_id column.
- dataset_tables has a column dataset_id which is a foreign key on datasets table on id column. dataset_tables has a column with name as table_name.
- entity_data table has the size of data stored for each entity. This table also has the table_name column.
- Use the table_name column in dataset_tables and entity_data for the join between two tables.
"""

prompt_template = PromptTemplate.from_template(
    f"""You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
    Unless the user specifies in the question a specific number of examples to obtain, query for at most {{top_k}} results using the LIMIT clause as per PostgreSQL. You can order the results to return the most informative data in the database.
    Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
    Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
    Pay attention to use CURRENT_DATE function to get the current date, if the question involves "today".

    Use the following format:

    Question: Question here
    SQLQuery: SQL Query to run
    SQLResult: Result of the SQLQuery
    Answer: Final answer here

    Only use the following context :
    {context}

    Question: {{input}}
    """
)

prompt = prompt_template.format(input="which entity has the most data stored?", top_k=5)

response = chain.invoke({"question": prompt})
print(response)
# Remove backticks from the response
cleaned_response = response.replace("```", "")
print(db.run(cleaned_response))

Available tables: ['annotations', 'annotations_account_codes', 'api_keys', 'ar_internal_metadata', 'async_job_results', 'async_jobs', 'audits', 'authorization_tokens', 'budgeting_allocation_plans', 'budgeting_base_line_period_attributes', 'budgeting_base_lines', 'budgeting_budget_artifacts', 'budgeting_budget_invites', 'budgeting_budget_phases', 'budgeting_budget_reference_data_sets', 'budgeting_budgets', 'budgeting_cost_driving_account_period_attributes', 'budgeting_cost_driving_accounts', 'budgeting_cost_pools', 'budgeting_events', 'budgeting_files', 'budgeting_itemized_entries', 'budgeting_operation_lines', 'budgeting_operations', 'budgeting_periods', 'budgeting_permitted_proposal_segment_codes', 'budgeting_proposal_details', 'budgeting_proposal_invites', 'budgeting_proposals', 'budgeting_pseudoids', 'budgeting_receiver_period_attributes', 'budgeting_receivers', 'budgeting_restricted_segments', 'budgeting_snapshot_coa_snapshots', 'budgeting_snapshot_denormalized_coas', 'change_log',

In [ ]:
# response = chain.invoke({"question": "Which state are customers most from?"})
# print(response)
print(db.run(response))

In [50]:
response = chain.invoke({"question": "Give me the top 5 customers who paid the highest shipping charges"})
print(response)
print(db.run(response))

SELECT "customerid", "firstname", "lastname", "shipping"
FROM customer
JOIN sale ON customer.customerid = sale.customer_id
ORDER BY "shipping" DESC
LIMIT 5;
[(129, 'Kurt', 'Small', Decimal('59.47')), (109, 'Elham', 'Mousavidin', Decimal('59.37')), (156, 'Selris', 'Mahabir', Decimal('59.03')), (176, 'Uchenna', 'Sr', Decimal('58.15')), (672, 'Matthew', 'Rowland', Decimal('55.53'))]
